# The Battle of Neighborhoods
   
   ###  Course: Applied Data Science Capstone
   
This notebook would contain Explaination and code for the coursera Capstone Project. 


   ### Adding Required Libraries to create required Environment

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

pd.set_option('display.expand_frame_repr', False)

First Print Statement !! :)


In [2]:
print('Hello Capstone Project Course!')


Hello Capstone Project Course!


# Segmenting and Clustering Neighborhoods in Toronto

## 1. Data scraping

#### Description: 
Code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe.


In [3]:
# define the dataframe columns
column_names = ['Postal_Code','Borough', 'Neighborhood'] 

# instantiate the dataframe
nbr = pd.DataFrame(columns=column_names)

Take a look at the empty dataframe to confirm that the columns are as intended.

In [4]:
nbr

,Postal_Code,Borough,Neighborhood


#### Web scraping
##### Importing Data From provided URL

In [5]:
#import the library used to query a website
from urllib.request import urlopen

#specify the url
wiki = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

#Query the website and return the wiki to the variable 'page'
page = urlopen(wiki)

#import the Beautiful soup functions to parse the data returned from the website
from bs4 import BeautifulSoup

#Parse the html in the 'page' variable, and store it in Beautiful Soup format
soup = BeautifulSoup(page, "lxml")


In [6]:
# Uncomment below to Print HTML of the Page
#print(soup.prettify()) 

#Get table 
right_table=soup.find('table', class_='wikitable sortable')
# uncomment below to print table
#right_table  

##### Extract the information to DataFrame: 
we iterate through each row (tr) and then assign each element of tr (td) to a variable and append it to a list.

In [7]:
#Generate lists
A=[]
B=[]
C=[]

for row in right_table.findAll("tr"):
    cells = row.findAll('td')
    if len(cells)==3: #Only extract table body not heading
        A.append(cells[0].find(text=True))
        B.append(cells[1].find(text=True))
        C.append(cells[2].find(text=True))

        
#Adding Data to our DataFrame
nbr['Postal_Code']=A
nbr['Borough']=B
nbr['Neighborhood']=C

#uncomment below to view dataframe nbr which contains data as retreived from the webpage in wiki
#nbr

####  Ignore cells with a borough that is Not assigned.
1. Compare with Each row if Borough is Not Assigned drop row.
2. Reset Index.

In [8]:
nbr = nbr.drop(nbr[nbr['Borough'].str.contains("Not assigned")==True].index, axis=0, inplace=False)

#Reset Index
nbr.index = pd.RangeIndex(len(nbr.index))

#Uncomment below to view Dataframe where not Assigned Boroughs have been dropped out.
#nbr               

#### If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
1. instantiate additional dataframe nbrNA
2. Compare with Each row if Neighborhood is Not Assigned add value from Borough.

In [9]:
# instantiate additional dataframe nbrNA
column_n = ['NBRVAL'] 
nbrNA = pd.DataFrame(columns=column_n)
D=['Not assigned'] 
nbrNA['NBRVAL'] = D

#Dataframe contains 'Not assigned' value used for Comparison.
#print(nbrNA)

nbr1=nbr
#nbr1


#Compare with Each row if Neighborhood is Not Assigned add value from Borough.
for row_index,row in nbr.iterrows():
    if((nbr.loc[row_index,['Neighborhood']].values.astype('str') == 'Not assigned') or (nbr.loc[row_index,['Neighborhood']].values.astype('str') == 'Not assigned\n')):
        #print('inside if')
        nbr1.loc[row_index,['Neighborhood']] = nbr1.loc[row_index,['Borough']].values.astype('str') 
        
#Reset Index
#nbr.index = pd.RangeIndex(len(nbr.index))

#Uncomment below to view Dataframe nbr1 which contains Neighborhoods which were initially 'Not Assigned' now containing Borough values
print('nbr1')
nbr1               

nbr1


,Postal_Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Queen's Park
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


##### More than one neighborhood can exist in one postal code area.

In [10]:
nbr2=nbr1.groupby('Postal_Code').agg({'Borough':'first',
                               'Neighborhood': ', '.join}).reset_index()
#Uncomment Below to view nbr2 which contains grouped Boroughs
nbr2

,Postal_Code,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood\n, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park\n, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West\n"
9,M1N,Scarborough,"Birch Cliff, Cliffside West\n"


#### Printing the number of rows of neighborhoods dataframe.

In [11]:
print(nbr2.shape)
print('nbr2')
nbr2

(103, 3)
nbr2


,Postal_Code,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood\n, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park\n, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West\n"
9,M1N,Scarborough,"Birch Cliff, Cliffside West\n"


## Filtering Neighborhood
Creating a copy of Dataframe.
Dropping neighborhoods which do not Belong to Toronto

In [12]:
# define the dataframe columns
column_names = ['Postal_Code','Borough', 'Neighborhood'] 
# instantiate the dataframe
neighbor = pd.DataFrame(columns=column_names)

neighbor = nbr2.drop(nbr2[nbr2['Borough'].str.contains("Toronto")==False].index, axis=0, inplace=False)

#Reset Index
neighbor.index = pd.RangeIndex(len(neighbor.index))
print('neighbor')
print(neighbor)

neighbor
   Postal_Code           Borough                                       Neighborhood
0          M4E      East Toronto                                        The Beaches
1          M4K      East Toronto                     The Danforth West\n, Riverdale
2          M4L      East Toronto                   The Beaches West\n, India Bazaar
3          M4M      East Toronto                                  Studio District\n
4          M4N   Central Toronto                                      Lawrence Park
5          M4P   Central Toronto                                 Davisville North\n
6          M4R   Central Toronto                               North Toronto West\n
7          M4S   Central Toronto                                       Davisville\n
8          M4T   Central Toronto                      Moore Park, Summerhill East\n
9          M4V   Central Toronto  Deer Park, Forest Hill SE\n, Rathnelly, South ...
10         M4W  Downtown Toronto                                   

In [13]:
#Keeping nbr_ungrp for ease of location calculations, using nbr1 by dropping Non Toronto Boroughs.
# define the dataframe columns
column_names = ['Postal_Code','Borough', 'Neighborhood'] 
# instantiate the dataframe
nbr_ungrp = pd.DataFrame(columns=column_names)

nbr_ungrp = nbr1.drop(nbr1[nbr1['Borough'].str.contains("Toronto")==False].index, axis=0, inplace=False)

#Reset Index
nbr_ungrp.index = pd.RangeIndex(len(nbr_ungrp.index))

#Uncomment below to view Dataframe
print(nbr_ungrp)

   Postal_Code           Borough                                       Neighborhood
0          M5A  Downtown Toronto                                       Harbourfront
1          M5A  Downtown Toronto                                        Regent Park
2          M5B  Downtown Toronto                                            Ryerson
3          M5B  Downtown Toronto                                   Garden District

4          M5C  Downtown Toronto                                     St. James Town
5          M4E      East Toronto                                        The Beaches
6          M5E  Downtown Toronto                                        Berczy Park
7          M5G  Downtown Toronto                                Central Bay Street

8          M6G  Downtown Toronto                                          Christie

9          M5H  Downtown Toronto                                           Adelaide
10         M5H  Downtown Toronto                                            

### Capturing Longitude And Latitude info of Locations
using the Geocoder Python package instead: https://geocoder.readthedocs.io/index.html

In [14]:
# uncomment this line if you haven't completed the Foursquare API lab
!conda install -c conda-forge geopy --yes 
# convert an address into latitude and longitude values
import time
from geopy.geocoders import Nominatim 

Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
geopy                     1.17.0                     py_0    conda-forge


#### Looping through locations to fetch Latitude and Longitude.

In [15]:
geolocator = Nominatim(user_agent="my_app app123")
#print('here')

for row_index, item in nbr_ungrp.iterrows():
    
    list1 = nbr_ungrp.loc[[row_index],['Neighborhood']].values.astype('str')
    loc = ' , Toronto, Ontario, Canada'
    list1.astype('str')
    list1 = np.append(list1, loc)
    latitude = None
    longitude = None
    location = None
    
    location = geolocator.geocode(list1 , limit = 15)
    #print(loc)
    #print(row_index)
    time.sleep(5)
    if(location is not None):
        #print(location)
        #print("location Address : ")
        #print(location.address)
        #print('latitude, longitude : ')
        #print((location.latitude, location.longitude))
        nbr_ungrp.loc[nbr_ungrp.index[row_index], 'Latitude'] = location.latitude
        nbr_ungrp.loc[nbr_ungrp.index[row_index], 'Longitude'] = location.longitude
        #print(nbr_ungrp[row_index])    

print(nbr_ungrp)


   Postal_Code           Borough                                       Neighborhood   Latitude  Longitude
0          M5A  Downtown Toronto                                       Harbourfront  43.640080 -79.380150
1          M5A  Downtown Toronto                                        Regent Park  43.660706 -79.360457
2          M5B  Downtown Toronto                                            Ryerson  43.621573 -79.559130
3          M5B  Downtown Toronto                                   Garden District
  43.656502 -79.377128
4          M5C  Downtown Toronto                                     St. James Town  43.669403 -79.372704
5          M4E      East Toronto                                        The Beaches  43.671024 -79.296712
6          M5E  Downtown Toronto                                        Berczy Park  43.648001 -79.375385
7          M5G  Downtown Toronto                                Central Bay Street
        NaN        NaN
8          M6G  Downtown Toronto              

# Neighborhood analysis#
# Importing Libraries for Foursqaue to analyse Neighborhood.
    


In [16]:
import json # library to handle JSON files

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

# Importing to use the Foursquare API lab
!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

print('Foursquare and map plotting Libraries imported.')


Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.0-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge

altair-2.2.2-p 100% |################################| Time: 0:00:00   1.25 MB/s
branca-0.3.0-p 100% |################################| Time: 0:00:00  30.93 MB/s
vincent-0.4.4- 100% |################################| Time: 0:00:00  36.35 MB/s
folium-0.5.0-p 100% |################################| Time: 0:00:00  39.60 MB/s
Foursquare and map plotting Libraries imported.


#### Number of Boroughs and Neighborhoods

In [17]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(nbr_ungrp['Borough'].unique()),
        nbr_ungrp.shape[0]
    )
)

#Dropping NANs
nbr_ungrp.dropna(inplace =True)
#Reset Index
nbr_ungrp.index = pd.RangeIndex(len(nbr_ungrp.index))
nbr_ungrp


The dataframe has 4 boroughs and 74 neighborhoods.


,Postal_Code,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,Harbourfront,43.640080,-79.380150
1,M5A,Downtown Toronto,Regent Park,43.660706,-79.360457
2,M5B,Downtown Toronto,Ryerson,43.621573,-79.559130
3,M5B,Downtown Toronto,Garden District,43.656502,-79.377128
4,M5C,Downtown Toronto,St. James Town,43.669403,-79.372704
5,M4E,East Toronto,The Beaches,43.671024,-79.296712
6,M5E,Downtown Toronto,Berczy Park,43.648001,-79.375385
7,M6G,Downtown Toronto,Christie,43.664111,-79.418405
8,M5H,Downtown Toronto,Adelaide,43.650298,-79.380477
9,M5H,Downtown Toronto,King,43.648949,-79.377754


#### Using geopy library to get the latitude and longitude values of Toronto City.

In [18]:
address = 'Toronto, Ontario, Canada'

geolocator = Nominatim(user_agent="my-app123")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


## Create a map of Toronto with neighborhoods superimposed on top.

In [19]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(nbr_ungrp['Latitude'], nbr_ungrp['Longitude'], nbr_ungrp['Borough'], nbr_ungrp['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

## Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.
#### Define Foursquare Credentials and Version

In [20]:
# The code was removed by Watson Studio for sharing.

Your credentails:
CLIENT_ID: M5YWFTCLNYBN54LXSWEYYETLT2WWJM4OO1BNH2GD2NHFYBVM
CLIENT_SECRET:0DWUELCVOLNKHNFDYKFOYA4JTLXIGDSCXOTDNA40ZVHPEBML


### Let's explore the first neighborhood in our dataframe.
Get the neighborhood's details


In [21]:
nbr_ungrp.loc[0]
neighborhood_latitude = np.float(nbr_ungrp.loc[0,['Latitude']].values)
neighborhood_longitude =  np.float(nbr_ungrp.loc[0,['Longitude']].values)

#### Now, let's get the top 25 venues that are in Harbourfront within a radius of 500 meters.
First, let's create the GET request URL. Naming the URL url.

In [23]:
# The code was removed by Watson Studio for sharing.

https://api.foursquare.com/v2/venues/explore?&client_id=M5YWFTCLNYBN54LXSWEYYETLT2WWJM4OO1BNH2GD2NHFYBVM&client_secret=0DWUELCVOLNKHNFDYKFOYA4JTLXIGDSCXOTDNA40ZVHPEBML&v=20180605&ll=43.6400801,-79.3801495&radius=500&limit=100


####  get_category_type function

In [24]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

cleaning the json and structuring it into a pandas dataframe.

In [25]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Lake Ontario,Lake,43.639398,-79.379589
1,Harbour Square Park,Park,43.639253,-79.378395
2,Harbourfront,Neighborhood,43.639526,-79.380688
3,BeaverTails,Bakery,43.639899,-79.380197
4,Sharetea-Waterfront,Bubble Tea Shop,43.640176,-79.379606


 how many venues were returned by Foursquare?

In [26]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


### Exploring all Neighborhoods-


In [27]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [28]:
### Getting Toronto Venues


In [29]:
toronto_venues = getNearbyVenues(names=nbr_ungrp['Neighborhood'],
                                   latitudes=nbr_ungrp['Latitude'],
                                   longitudes=nbr_ungrp['Longitude']
                                  )




Harbourfront
Regent Park
Ryerson
Garden District

St. James Town
The Beaches
Berczy Park
Christie

Adelaide
King
Richmond

Dovercourt Village
Dufferin

Harbourfront East

Toronto Islands
Union Station
Little Portugal
Trinity
The Danforth West

Riverdale
Design Exchange
Toronto Dominion Centre
Brockton

Exhibition Place
Parkdale Village
The Beaches West

India Bazaar
Commerce Court
Lawrence Park
Roselawn

Davisville North

Forest Hill North
High Park
The Annex
Yorkville
Parkdale
Roncesvalles
Davisville

Harbord

University of Toronto
Runnymede
Swansea
Moore Park
Summerhill East

Chinatown
Grange Park
Kensington Market
Deer Park
Forest Hill SE

Rathnelly
South Hill
Summerhill West

CN Tower
Bathurst Quay

Harbourfront West

King and Spadina
South Niagara
Rosedale
Cabbagetown
St. James Town
First Canadian Place
Underground city
Church and Wellesley


Size of dataframe

In [30]:
print(toronto_venues.shape)
toronto_venues.head()

(3280, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Harbourfront,43.64008,-79.38015,Lake Ontario,43.639398,-79.379589,Lake
1,Harbourfront,43.64008,-79.38015,Harbour Square Park,43.639253,-79.378395,Park
2,Harbourfront,43.64008,-79.38015,Harbourfront,43.639526,-79.380688,Neighborhood
3,Harbourfront,43.64008,-79.38015,BeaverTails,43.639899,-79.380197,Bakery
4,Harbourfront,43.64008,-79.38015,Sharetea-Waterfront,43.640176,-79.379606,Bubble Tea Shop


Let's check how many venues were returned for each neighborhood

In [31]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Adelaide,100,100,100,100,100,100
Bathurst Quay,22,22,22,22,22,22
Berczy Park,100,100,100,100,100,100
Brockton,19,19,19,19,19,19
CN Tower,86,86,86,86,86,86
Cabbagetown,53,53,53,53,53,53
Chinatown,100,100,100,100,100,100
Christie,60,60,60,60,60,60
Church and Wellesley,92,92,92,92,92,92


#### Let's find out how many unique categories can be curated from all the returned venues

In [32]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 281 uniques categories.


## 3. Analyze Each Neighborhood

In [33]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Zoo,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport Service,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Café,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Costume Shop,Creperie,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Theater,Indonesian Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Korean Restaurant,Lake,Latin American Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Outdoor Supply Store,Pakistani Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Piano Bar,Pie Shop,Pizza Place,Platform,Playground,Plaza,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Pub,Racetrack,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Rock Climbing Spot,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Repair,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Stadium,Soup Place,South American Restaurant,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Summer Camp,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tattoo Parlor,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Track,Trail,Train Station,Tree,Tunnel,University,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio

In [34]:
toronto_onehot.shape

(3280, 281)

let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [35]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Zoo,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport Service,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Café,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Costume Shop,Creperie,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Theater,Indonesian Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Korean Restaurant,Lake,Latin American Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Outdoor Supply Store,Pakistani Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Piano Bar,Pie Shop,Pizza Place,Platform,Playground,Plaza,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Pub,Racetrack,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Rock Climbing Spot,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Repair,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Stadium,Soup Place,South American Restaurant,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Summer Camp,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tattoo Parlor,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Track,Trail,Train Station,Tree,Tunnel,University,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio

In [36]:
toronto_grouped.shape

(62, 281)

Let's print each neighborhood along with the top 5 most common venues

In [37]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide----
         venue  freq
0  Coffee Shop  0.08
1         Café  0.06
2        Hotel  0.05
3   Restaurant  0.05
4    Gastropub  0.04


----Bathurst Quay
----
         venue  freq
0  Coffee Shop  0.18
1         Café  0.14
2         Park  0.09
3          Pub  0.05
4       Tunnel  0.05


----Berczy Park----
         venue  freq
0  Coffee Shop  0.09
1         Café  0.06
2        Hotel  0.04
3       Bakery  0.04
4   Restaurant  0.04


----Brockton
----
                   venue  freq
0                    Bar  0.16
1            Coffee Shop  0.11
2  Vietnamese Restaurant  0.11
3                   Park  0.11
4  Portuguese Restaurant  0.05


----CN Tower----
                venue  freq
0         Coffee Shop  0.09
1               Hotel  0.09
2         Pizza Place  0.05
3  Italian Restaurant  0.05
4            Aquarium  0.05


----Cabbagetown----
                venue  freq
0          Restaurant  0.11
1         Coffee Shop  0.08
2                Café  0.08
3   Indian Restaurant  0.06
4  

#### Let's put that into a *pandas* dataframe
First, let's write a function to sort the venues in descending order.

In [38]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 7 venues for each neighborhood.

In [39]:
num_top_venues = 7

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue
0,Adelaide,Coffee Shop,Café,Restaurant,Hotel,Gastropub,Italian Restaurant,American Restaurant
1,Bathurst Quay,Coffee Shop,Café,Park,Diner,Caribbean Restaurant,Sculpture Garden,Sushi Restaurant
2,Berczy Park,Coffee Shop,Café,Hotel,Bakery,Restaurant,Gym,Gastropub
3,Brockton,Bar,Coffee Shop,Vietnamese Restaurant,Park,Bakery,Light Rail Station,Dive Bar
4,CN Tower,Hotel,Coffee Shop,Aquarium,Pizza Place,Italian Restaurant,Scenic Lookout,Sports Bar
5,Cabbagetown,Restaurant,Coffee Shop,Café,Indian Restaurant,Beer Store,Pizza Place,Bakery
6,Chinatown,Café,Chinese Restaurant,Bar,Vegetarian / Vegan Restaurant,Mexican Restaurant,Vietnamese Restaurant,Coffee Shop
7,Christie,Korean Restaurant,Coffee Shop,Indian Restaurant,Japanese Restaurant,Cocktail Bar,Ice Cream Shop,Dessert Shop
8,Church and Wellesley,Japanese Restaurant,Coffee Shop,Sushi Restaurant,Burger Joint,Gay Bar,Café,Restaurant
9,Commerce Court,Coffee Shop,Café,Hotel,Restaurant,American Restaurant,Gastropub,Steakhouse


## 4. Cluster Neighborhoods
Run *k*-means to cluster the neighborhood into 4 clusters.

In [40]:
# set number of clusters
kclusters = 4

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood',1)
#print(toronto_grouped_clustering)
#print(toronto_grouped)
# run k-means clustering
kmeans = KMeans(init = "k-means++", n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
labels = kmeans.labels_[0:63] 
print(labels)

[3 3 3 3 3 0 0 0 0 3 0 0 0 3 3 0 0 3 1 1 0 0 3 3 3 3 0 0 0 3 0 0 3 2 0 0 0
 3 3 0 0 0 0 0 0 0 0 3 0 0 0 0 0 0 0 3 0 0 0 3 3 0]


Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [41]:
toronto_merged = nbr_ungrp
print(toronto_merged.shape)
labels = np.append(labels,labels[0])
print(labels.shape)
# add clustering labels
toronto_merged['Cluster Labels'] = labels.tolist()

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

(63, 5)
(63,)


,Postal_Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue
0,M5A,Downtown Toronto,Harbourfront,43.640080,-79.380150,3,Coffee Shop,Café,Hotel,Boat or Ferry,Pizza Place,Sushi Restaurant,Brewery
1,M5A,Downtown Toronto,Regent Park,43.660706,-79.360457,3,Coffee Shop,Thai Restaurant,Pharmacy,Grocery Store,Food Truck,Electronics Store,Sushi Restaurant
2,M5B,Downtown Toronto,Ryerson,43.621573,-79.559130,3,Sandwich Place,Pet Store,Discount Store,Burger Joint,Furniture / Home Store,Gift Shop,Arts & Crafts Store
3,M5B,Downtown Toronto,Garden District,43.656502,-79.377128,3,Coffee Shop,Clothing Store,Restaurant,Café,Cosmetics Shop,Theater,Movie Theater
4,M5C,Downtown Toronto,St. James Town,43.669403,-79.372704,3,Coffee Shop,Pizza Place,Restaurant,Café,Indian Restaurant,Hostel,Food & Drink Shop


Finally, let's visualize the resulting clusters

In [42]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 5. Examine Clusters
#### Cluster 1

In [43]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue
5,East Toronto,0,Beach,Bar,Breakfast Spot,Japanese Restaurant,Coffee Shop,Pub,Tea Room
6,Downtown Toronto,0,Coffee Shop,Café,Hotel,Bakery,Restaurant,Gym,Gastropub
7,Downtown Toronto,0,Korean Restaurant,Coffee Shop,Indian Restaurant,Japanese Restaurant,Cocktail Bar,Ice Cream Shop,Dessert Shop
8,Downtown Toronto,0,Coffee Shop,Café,Restaurant,Hotel,Gastropub,Italian Restaurant,American Restaurant
10,Downtown Toronto,0,Coffee Shop,Park,Gym / Fitness Center,Bakery,Italian Restaurant,Mediterranean Restaurant,Spa
11,West Toronto,0,Café,Bus Line,Restaurant,Pizza Place,Bar,Juice Bar,Coffee Shop
12,West Toronto,0,Bar,Bakery,Café,Coffee Shop,Beer Store,Clothing Store,Cocktail Bar
15,Downtown Toronto,0,Coffee Shop,Café,Hotel,Deli / Bodega,Restaurant,Italian Restaurant,Sports Bar
16,West Toronto,0,Coffee Shop,Bar,Café,Bakery,Cocktail Bar,Restaurant,Vietnamese Restaurant
20,Downtown Toronto,0,Coffee Shop,Café,Hotel,American Restaurant,Restaurant,Gastropub,Seafood Restaurant


#### Cluster 2

In [44]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue
18,East Toronto,1,Coffee Shop,Pharmacy,Optical Shop,Middle Eastern Restaurant,Mexican Restaurant,Metro Station,Fish & Chips Shop
19,East Toronto,1,Vietnamese Restaurant,Chinese Restaurant,Bakery,Grocery Store,Light Rail Station,Baseball Field,Dim Sum Restaurant


#### Cluster 3

In [45]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue
33,Central Toronto,2,Pizza Place,Sandwich Place,Café,Mexican Restaurant,Vegetarian / Vegan Restaurant,French Restaurant,Italian Restaurant


#### Cluster 4

In [46]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue
0,Downtown Toronto,3,Coffee Shop,Café,Hotel,Boat or Ferry,Pizza Place,Sushi Restaurant,Brewery
1,Downtown Toronto,3,Coffee Shop,Thai Restaurant,Pharmacy,Grocery Store,Food Truck,Electronics Store,Sushi Restaurant
2,Downtown Toronto,3,Sandwich Place,Pet Store,Discount Store,Burger Joint,Furniture / Home Store,Gift Shop,Arts & Crafts Store
3,Downtown Toronto,3,Coffee Shop,Clothing Store,Restaurant,Café,Cosmetics Shop,Theater,Movie Theater
4,Downtown Toronto,3,Coffee Shop,Pizza Place,Restaurant,Café,Indian Restaurant,Hostel,Food & Drink Shop
9,Downtown Toronto,3,Coffee Shop,Hotel,Café,Restaurant,American Restaurant,Gastropub,Cocktail Bar
13,Downtown Toronto,3,Coffee Shop,Café,Hotel,Boat or Ferry,Pizza Place,Sushi Restaurant,Brewery
14,Downtown Toronto,3,Scenic Lookout,American Restaurant,Beach,Farm,Beer Garden,Filipino Restaurant,Fast Food Restaurant
17,West Toronto,3,Clothing Store,Coffee Shop,American Restaurant,Café,Cosmetics Shop,Tea Room,Chinese Restaurant
22,West Toronto,3,Bar,Coffee Shop,Vietnamese Restaurant,Park,Bakery,Light Rail Station,Dive Bar
